This notebook bins casts differently and calculates dominant water properties slightly differently than the previous NB.

In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
from sets import Set
import calendar
import seaborn as sns
import csvWOD
import falkor_ctd
import copy
import seawater as SW
import os
from mpl_toolkits.basemap import Basemap

# Switch to unbuffered output
# We create a class that overrides the write function of a stream object.
# In this write function, we call the super write, then we also flush
# the stream -- this has the effect of forcing output to the screen after 
# every write.
class Unbuffered(object):
    def __init__(self, stream):
        self.stream = stream
    def write(self, data):
        self.stream.write(data)
        self.stream.flush()
    def __getattr__(self, attr):
        return getattr(self.stream, attr)

import sys
sys.stdout = Unbuffered(sys.stdout)

sns.set()
sns.set_style("darkgrid")
sns.set_context("poster")

%matplotlib inline

In [3]:
def conv_falkor_to_csvWOD(thedata,index,station,latitude,longitude):
    ret_dat = {}
    
    ret_dat['Station'] = station
    ret_dat['Latitude'] = latitude
    ret_dat['Longitude'] = longitude
    ret_dat['Temperatur'] = list(thedata[index['Temperature']])
    ret_dat['Salinity'] = list(thedata[index['Salinity']])
    ret_dat['Oxygen'] = list(thedata[index['Oxygen']])
    #ret_dat['Fluorescence'] = list(thedata[index['Fluorescence']])
    #ret_dat['Turbidity'] = list(thedata[index['Turbidity']])
    ret_dat['Year'] = 2013
    ret_dat['Month'] = 8
    ret_dat['Units'] = ['degrees C','psu','g/kg','m']#,'','']
    ret_dat['Variables'] = ['Temperatur','Salinity','Oxygen','Depth']#,'Fluorescence','Turbidity']
    
    # convert from pressure to depth
    p = thedata[index['Pressure']]
    d = []
    for ii in p:
        d.append(SW.dpth(ii,latitude))
    
    ret_dat['Depth'] = list(d)
    
    return ret_dat

def load_falkor_dat():
    filenames = [f for f in os.listdir(falkor_ctd.DIRECTORY) if (f.startswith('2013-57-00') and f.endswith('avg'))]

    all_casts = []
    for count,filename in enumerate(sorted(filenames)):
        thedata,index,station,lat,lon = falkor_ctd.falkor_ctd_dat(filename)
         
        #print [('[',k,']') for k in lat.split(' ')]
        if len(lat.split(' ')[3]) > 0:
            latitude = float(lat.split(' ')[1]) + float(lat.split(' ')[3])/60.0
        else:
            latitude = float(lat.split(' ')[1]) + float(lat.split(' ')[4])/60.0
        
        #print [('[',k,']') for k in lon.split(' ')]
        if len(lon.split(' ')[2]) > 0:
            longitude = -(float(lon.split(' ')[0]) + float(lon.split(' ')[2])/60.0)
        else:
            longitude = -(float(lon.split(' ')[0]) + float(lon.split(' ')[3])/60.0)
        
        all_casts.append(conv_falkor_to_csvWOD(thedata,index,station,latitude,longitude))
        
    return all_casts
    
def load_NOAA_dat():
    cast_inf = []

    FILES = ['../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.CTD.csv',
             '../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.CTD2.csv',
             '../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.CTD3.csv',
             '../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.CTD4.csv',
             '../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.CTD5.csv'
             #'../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.OSD.csv'
            ]

    for fn in FILES:
        print '-----------------------------------------------------------------'
        print fn
        print '-----------------------------------------------------------------'
        print '> Parsing'
        tmp_casts = []
        tmp_casts = csvWOD.read_casts(fn)
        print '> Done parsing'
        print '> Regionally filtering' # likely don't need this filter 
        tmp_casts = csvWOD.filter_region(tmp_casts,-160,-120,15,60)
        tmp_casts = csvWOD.filter_variables(tmp_casts,['Temperatur','Salinity','Depth'])
        print '> Done filtering'
        print '> Appending new casts'
        cast_inf.extend(tmp_casts)
        print '> Done' 
        print ''

    print 'Total casts: ', len(cast_inf)
    return cast_inf

def load_all_data():
    print '========================================='
    print ''
    print '            LOADING FALKOR DATA'
    print ''
    print '========================================='
    
    FALKOR_CAST = load_falkor_dat()
    
    print '========================================='
    print ''
    print '            LOADING NOAA DATA'
    print ''
    print '========================================='
    
    NOAA_CAST = load_NOAA_dat()
        
    print 'DONE'
    NOAA_CAST.extend(FALKOR_CAST)
    
    cast_inf = copy.copy(NOAA_CAST)    
    return cast_inf

So at this point, we have all cast info loaded in.

In [4]:
cast_inf = load_all_data()


            LOADING FALKOR DATA

>  2013-57-0002.avg
>  2013-57-0003.avg
>  2013-57-0004.avg
>  2013-57-0005.avg
>  2013-57-0006.avg
>  2013-57-0007.avg
>  2013-57-0008.avg
>  2013-57-0009.avg
>  2013-57-0010.avg
>  2013-57-0011.avg
>  2013-57-0012.avg
>  2013-57-0013.avg
>  2013-57-0014.avg
>  2013-57-0015.avg
>  2013-57-0016.avg
>  2013-57-0017.avg
>  2013-57-0018.avg
>  2013-57-0019.avg
>  2013-57-0020.avg
>  2013-57-0021.avg
>  2013-57-0022.avg
>  2013-57-0023.avg
>  2013-57-0024.avg
>  2013-57-0025.avg
>  2013-57-0026.avg
>  2013-57-0027.avg
>  2013-57-0028.avg
>  2013-57-0029.avg
>  2013-57-0030.avg
>  2013-57-0031.avg
>  2013-57-0032.avg
>  2013-57-0033.avg
>  2013-57-0034.avg
>  2013-57-0035.avg
>  2013-57-0036.avg
>  2013-57-0037.avg
>  2013-57-0038.avg
>  2013-57-0039.avg
>  2013-57-0040.avg

            LOADING NOAA DATA

-----------------------------------------------------------------
../../5_WOD13_Data/WOD13_004/ocldb1432579402.10924.CTD.csv
-----------------------------